# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial use [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train a LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [6]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

Install openai package

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [9]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens with different performance and cost. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responsed content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [10]:
messages = [{"role": "user", "content": "Who is the U.S President?"}]

print(openai_help(messages))

As of my last update, Joe Biden is the President of the United States. However, please verify with a current source, as this information may have changed.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [11]:
messages = [
    {"role": "system", "content": "You are a zealoted supporter of Silicon Valley tech firms"},
    {"role": "user", "content": "What is the best state to start a technology company?"}
    ]

print(openai_help(messages, temperature = 0.8))

When considering the best state to start a technology company, it's hard to overlook the powerhouse that is California, particularly Silicon Valley. California is home to a vast ecosystem of established tech giants, innovative startups, and venture capital firms, providing a fertile ground for networking, investment opportunities, and talent acquisition. The culture of innovation and risk-taking is deeply embedded in the region, making it an ideal environment for tech entrepreneurs.

That being said, other states are also emerging as attractive options for tech startups. Texas, with cities like Austin, offers a vibrant tech scene with a lower cost of living and doing business compared to California. The state is seeing a growing influx of tech talent and investment due to its favorable business climate and tax policies.

Washington, home to tech behemoths like Microsoft and Amazon, also presents a strong case with its robust infrastructure and access to skilled professionals. Similarly

Add assistant messages to teach LLM what `##` is.

In [12]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [13]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

Load some Twitter data from a text file. 

In [14]:
f =open("tweet_collection.txt", "r")
tweets = f.read()
f.close()

### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [15]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss various political topics related to an upcoming election. They include a pledge to donate to Kamala Harris and other Democratic candidates, a commentary on the local versus national nature of the election, concerns about voter fraud involving Glenn Youngkin's son, and criticism of Kamala Harris's campaign strategy. There are also mentions of conspiracy theories about election rigging by Republicans, the significance of the election, and skepticism about accepting the election outcome if Trump loses. Additionally, there is a mention of an election ploy and a reference to the Chief Election Commissioner's statement on election results in Haryana.


In [16]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

Tweets focus on election concerns, voter fraud, Kamala Harris's strategy, and conspiracy theories, highlighting political tensions.


In [17]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter},
                                    focuse on how people discuss about AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

The tweets focus on political discussions, with no direct mention of AI. Conversations revolve around election concerns, voter fraud, and political strategies, highlighting the contentious nature of the upcoming election.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [18]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)

In [19]:
for tweet in tweets.split('\n')[60:70]:
    flag_help(tweet)

===
violence
===
harassment


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [22]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimitered by {delimiter} into Chineese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

“转发 @MikeNellis: 我说过在选举日之前我会再做一次，所以我们开始吧……\n\n我将捐款给 @KamalaHarris 和其他民主党候选人……”
所有政治都是地方性的？这次选举不是这样 https://t.co/rzmTMWE3dc https://t.co/NYCmAYPTb9
“转发 @NotHoodlum: 格伦·杨金对选民欺诈非常关注。然而，当他17岁的儿子被抓到时，他却只字未提……”
"立即阅读：‘她为什么不努力工作’：政治专家质疑哈里斯的策略——哈里斯的低调日程引起政治专家的担忧，他们质疑在距离选举不到三周的时间里，她为何缺乏紧迫感……https://t.co/Mvhejh8Ajo"
“转推 @CollinRugg: 最新消息：MSNBC的Joy Reid提出新的阴谋论，指责*共和党人*操纵选举，称美国已经被…”
RT @BillieJeanKing: 我们距离可能是美国历史上最重要的选举还有整整3周的时间。 \n\n无论如何强调这一点都不为过…
RT @CallForCongress: 这是一个选举策略\n\n立即实施武器禁运
“转发 @ScottAdamsSays: 如果特朗普在当前条件下输掉选举，我将不接受选举结果的有效性。\n\n我们距离……”
"@GuntherEagleman 我不知道，当她赢得选举时你会离开吗？"
RT @amarDgreat: @sardesairajdeep @ECISVEEP 首席选举专员在新闻发布会上：\n\n在哈里亚纳邦结果公布日：\n我们的计票开始…


In [23]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimitered by {delimiter} in the tone of 400 year old chess grandmaster """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

"RT @MikeNellis: Verily, I proclaimed that I would undertake this endeavor once more ere the Day of Election doth arrive, thus I proceed... \n\nI shall bestow my coin upon @KamalaHarris and those who stand beneath her banner in the downballot D..."
Verily, it hath been said that all matters of governance are of a local nature. Yet, in this most peculiar election, such a notion doth not hold true. Behold, the affairs of state transcendeth the boundaries of locality, weaving a tapestry of influence that doth stretch far and wide. Prithee, gaze upon the chronicles herein for further enlightenment: https://t.co/rzmTMWE3dc https://t.co/NYCmAYPTb9
"RT @NotHoodlum: Verily, Glenn Youngkin doth express great concern regarding the malfeasance of voter fraud. Yet, he remaineth silent as the grave when his own progeny, a mere stripling of seventeen summers, was ensnared in such a scandalous act…"
"Prithee, lend thine ear to this discourse: 'Why Doth She Not Toil Diligently?': Sages of the Politica

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [23]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democra or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "positive",
  "emotion": "supportive",
  "mentioned": ["MikeNellis", "KamalaHarris"],
  "support": "Democrat"
}
{
  "sentiment": "neutral",
  "emotion": "indifference",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["Glenn Youngkin"],
  "support": "Democrat"
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["Harris"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "suspicion",
  "mentioned": ["Collin Rugg", "Joy Reid"],
  "support": "Democrat"
}
{
  "sentiment": "neutral",
  "emotion": "anticipation",
  "mentioned": ["BillieJeanKing"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "cynicism",
  "mentioned": ["CallForCongress"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "distrust",
  "mentioned": ["ScottAdamsSays", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned

In [24]:
tweet_sample = tweets.split("\n")[:10]

messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Donations",
    "Local vs National Politics",
    "Voter Fraud Concerns",
    "Kamala Harris's Campaign Strategy",
    "Election Conspiracy Theories",
    "Significance of Upcoming Election",
    "Election Tactics and Strategies",
    "Election Outcome Acceptance",
    "Election Predictions",
    "Election Process and Results"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [29]:
tweet_sample = tweets.split("\n")[:100]
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_sample):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democrat or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 100/100 [01:44<00:00,  1.05s/it]


In [26]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democrat and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democat count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democrat count": 20,
  "Republican count": 34,
  "Kamala Harris": {
    "sentiments": ["positive", "negative", "neutral"],
    "emotions": ["supportive", "frustration", "satisfaction", "admiration", "concern", "criticism"]
  },
  "Donald Trump": {
    "sentiments": ["negative", "neutral"],
    "emotions": ["anger", "concern", "informative", "outrage", "distrust"]
  },
  "Biden": {
    "sentiments": ["negative", "positive"],
    "emotions": ["frustration", "hopeful"]
  },
  "MikeNellis": {
    "sentiments": ["positive"],
    "emotions": ["supportive"]
  },
  "Glenn Youngkin": {
    "sentiments": ["negative"],
    "emotions": ["concern", "frustration"]
  },
  "ScottAdamsSays": {
    "sentiments": ["negative"],
    "emotions": ["distrust"]
  },
  "Joe Rogan": {
    "sentiments": ["negative"],
    "emotions": ["frustration"]
  },
  "SenJohnKennedy": {
    "sentiments": ["negative"],
    "emotions": ["anger", "betrayal"]
  },
  "MTGrepp": {
    "sentiments": ["neutral", "negative"],
  

In [27]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are an election chabot anwser user questions based on the tweets {delimiter} to answer user questions,
                                {delimiter}{tweet_sample}{delimiter} 
                                if user mentioned a people name in the {analysis_summary} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [28]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  Who is the top candidate according to these tweets?


Chatbot: The tweets and the sentiment analysis provided do not explicitly mention who the "top candidate" might be. However, they highlight expressions of both supportive and critical sentiments toward various political figures like Kamala Harris, Donald Trump, and others. Kamala Harris and Donald Trump appear frequently, with emotions ranging from supportive to frustration and anger, but the data doesn't indicate a clear "top candidate" based on these tweets.


You:  Who does this data suggest may win the 2024 election?


Chatbot: The data from the tweets provided does not make clear predictions about who may win the 2024 election. It captures only sentiments and emotions expressed about various political figures, indicating diverse opinions rather than a consensus or prediction. With expressions of both supportive and critical sentiments toward major figures like Kamala Harris and Donald Trump, it suggests a divided electorate rather than a clear favorite for the 2024 election.


You:  Who is mean most disliked candidate?


...

You:  Is the transgender community mentioned in these tweets?


Chatbot: Yes, the transgender community is indirectly mentioned in a tweet where Kamala Harris is criticized for allegedly using taxpayer money for a convicted felon's sex change, although the context and details are not fully provided. Here's the relevant portion of the tweet:

"BretBaier This will be the biggest pillow fight of the election process. She was indeed the Border Czar, she lied to us about the health of President Biden, used tax payer $ to pay for a convicted felons sex change, no wars when Trump was President and what is her position on the freedom…"

(Note: This tweet reflects the opinions and claims made by the user and may not represent verified facts.)


You:  Do any of these tweets violate Twitters/X's content guidelines?


Chatbot: To determine whether any specific tweets violate Twitter's/X's content guidelines, it is important to closely analyze the content for any violations related to hateful conduct, harassment, misinformation, or other prohibited behaviors outlined in Twitter's rules. 

Based on the tweets provided, some potential areas of concern could include:
- Claims of election rigging or fraud without evidence, especially if spreading false information about the integrity of elections.
- Disparaging or misleading statements about individuals or groups, which could potentially fall under harassment or hateful conduct, especially if they target specific identities or spread harmful stereotypes.

However, it would require a thorough review by Twitter's Trust and Safety team to definitively conclude if any specific tweets violate their guidelines, as they consider context, intent, and impact when evaluating potential violations.


KeyboardInterrupt: Interrupted by user

## Reference
- *“ChatGPT Prompt Engineering for Developers - DeepLearning.AI.”* n.d. DeepLearning.AI - Learning Platform. Accessed October 17, 2024. https://learn.deeplearning.ai/courses/chatgpt-prompt-eng/lesson/1/introduction.

- *“Building Systems with the ChatGPT API - DeepLearning.AI.”* n.d. DeepLearning.AI - Learning Platform. Accessed October 17, 2024. https://learn.deeplearning.ai/courses/chatgpt-building-system/lesson/1/introduction.

- *“OpenAI Documents.”* n.d. OpenAI Platform. Accessed October 18, 2024. https://platform.openai.com.
